# Example: Online Stochastic Bandit Algorithms for Portfolio Rebalancing
In this example, we'll propose a stochastic multi-armed bandit approach to portfolio rebalancing where a bandit algorithm selects which assets to include in the portfolio, and an optimal Utility Maximization (UM) approach is used to determine the optimal weights for the selected assets. Thus, we simultaneously address the asset selection and weight allocation problems in portfolio management.

> __Learning Objectives__
> 
> By the end of this example, you should be able to:
> * __Apply multi-armed bandit algorithms__ to explore different portfolio compositions by balancing exploration of uncertain subsets with exploitation of known high-utility combinations.
> * __Formulate and solve utility maximization problems__ using Cobb-Douglas utility functions to determine optimal asset share counts for a selected portfolio, with the understanding that this has an analytical solution for asset selections.
> * __Integrate bandit selection with utility maximization__ to demonstrate how these two approaches work together to simultaneously address asset selection and weight allocation problems in portfolio management.

This material will be presented at the upcoming [INFORMS Annual Meeting 2025](https://meetings.informs.org/wordpress/annual/) in Atlanta, GA on October 26-29, 2025. Let's get started!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> __Include:__ The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [3]:
include(joinpath(@__DIR__, "Include-informs.jl")); # include the Include.jl file

For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

### Data
We gathered daily open-high-low-close (OHLC) data for each firm in the [S&P 500](https://en.wikipedia.org/wiki/S%26P_500) from `01-03-2025` until `10-17-2025`, along with data for several exchange-traded funds and volatility products during that time period.

Let's load the `original_dataset::DataFrame` by calling [the `MyTestingMarketDataSet()` function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/data/#VLQuantitativeFinancePackage.MyTestingMarketDataSet).

In [6]:
original_out_of_sample_dataset = MyTestingMarketDataSet() |> x-> x["dataset"] # load the original dataset (testing)

Dict{String, DataFrame} with 482 entries:
  "NI"   => 182×8 DataFrame…
  "EMR"  => 182×8 DataFrame…
  "CTAS" => 182×8 DataFrame…
  "HSIC" => 182×8 DataFrame…
  "KIM"  => 182×8 DataFrame…
  "PLD"  => 182×8 DataFrame…
  "IEX"  => 182×8 DataFrame…
  "BAC"  => 182×8 DataFrame…
  "CBOE" => 182×8 DataFrame…
  "EXR"  => 182×8 DataFrame…
  "NCLH" => 182×8 DataFrame…
  "CVS"  => 182×8 DataFrame…
  "DRI"  => 182×8 DataFrame…
  "DTE"  => 182×8 DataFrame…
  "ZION" => 182×8 DataFrame…
  "AVY"  => 182×8 DataFrame…
  "EW"   => 182×8 DataFrame…
  "EA"   => 182×8 DataFrame…
  "NWSA" => 182×8 DataFrame…
  "BBWI" => 182×8 DataFrame…
  "CAG"  => 182×8 DataFrame…
  "GPC"  => 182×8 DataFrame…
  "FCX"  => 182×8 DataFrame…
  "GILD" => 182×8 DataFrame…
  "INFO" => 182×8 DataFrame…
  ⋮      => ⋮

Not all tickers in our dataset have the maximum number of trading days for various reasons, such as acquisition or delisting events. Let's collect only those tickers with the maximum number of trading days.

First, let's compute the number of records for a firm that we know has the maximum value, e.g., `AAPL`, and save that value in the `maximum_number_trading_out_of_sample_days::Int64` variable:

In [8]:
maximum_number_trading_out_of_sample_days = original_out_of_sample_dataset["AAPL"] |> nrow; # maximum number of trading days in our dataset 

Now, let's iterate through our data and collect only tickers with `maximum_number_trading_out_of_sample_days` records. Save that data in the `dataset::Dict{String,DataFrame}` variable:

In [10]:
dataset = let

    # initialize -
    dataset = Dict{String, DataFrame}();

    # iterate through the dictionary; we can't guarantee a particular order
    for (ticker, data) ∈ original_out_of_sample_dataset  # we get each (K, V) pair!
        if (nrow(data) == maximum_number_trading_out_of_sample_days) # check if ticker has maximum trading days
            dataset[ticker] = data;
        end
    end
    dataset; # return
end;

Let's get a list of the firms in our cleaned dataset and sort them alphabetically. We store the sorted firm ticker symbols in the `list_of_tickers_price_data::Array{String,1}` variable:

In [12]:
list_of_tickers_price_data = keys(dataset) |> collect |> sort;

Finally, let's load the single index model parameters that we computed in the previous example. We'll store this data in the `sim_model_parameters::Dict{String,NamedTuple}` variable. In addition, we return a few other useful variables, such as the historical market growth rate, the mean and variance of the market growth, etc.

In [14]:
sim_model_parameters,Gₘ,Ḡₘ, Varₘ = let

    # initialize -
    path_to_sim_model_parameters = joinpath(_PATH_TO_DATA,"SIMs-SPY-SP500-01-03-14-to-12-31-24.jld2");
    sim_model_parameters = JLD2.load(path_to_sim_model_parameters);
    parameters = sim_model_parameters["data"]; # return

    Gₘ = sim_model_parameters["Gₘ"]; # Get the past market growth rate 
    Ḡₘ = sim_model_parameters["Ḡₘ"]; # mean of market growth rates
    Varₘ = sim_model_parameters["Varₘ"]; # variance of market growth

    # return -
    parameters,  Gₘ , Ḡₘ, Varₘ;
end;

Now let's get a list of all tickers for which we have single index model parameters:

In [16]:
tickers_that_we_sim_sim_data_for = keys(sim_model_parameters) |> collect |> sort;

We need to use only the tickers for which we have both price data and SIM parameters. We'll compute [the intersection of the two lists](https://docs.julialang.org/en/v1/base/collections/#Base.intersect) and store the result in the `list_of_tickers::Array{String,1}` variable:

In [18]:
list_of_tickers = intersect(tickers_that_we_sim_sim_data_for, list_of_tickers_price_data);

Finally, we need to compute the projection matrix `X̄::Array{Float64,2}` = $(\mathbf{X}^{\top}\mathbf{X})^{-1}\mathbf{X}^{\top}$ (which will be handy for later) where $\mathbf{X}$ is the market factor matrix:

In [20]:
X̄ = let
    
    
    Rₘ = Gₘ; # get Gₘ from the sim model parameters
    max_length = length(Rₘ);
    X = [ones(max_length) Rₘ];

    a = inv(transpose(X)*X)*transpose(X)
    a;
end

2×2766 Matrix{Float64}:
  0.000367293  0.000355057  0.0003607   …   0.000382956   0.000370094
 -5.4289e-5    6.10402e-5   7.85216e-6     -0.000201909  -8.06852e-5

### Constants
Finally, let's set some constants we'll use later in this notebook. The comments describe the constants, their units, and permissible values:

In [22]:
Δt = (1/252); # time-step
total_number_of_tickers = length(list_of_tickers); # how many tickers do we have in my dataset?
investment_budget = 10000.0; # initial budget of the agent
risk_free_rate = 0.0418; # risk free rate
μₘ =  Ḡₘ; # expected growth for SPY
B = investment_budget; # TODO: Budget for portfolio. We can change this later if we want
ϵ = 0.001; # hyperparameter: minimum share number for each asset
α::Float64 = 0.80; # learning rate for moving average calculation
Tₘ::Int64 = 10; # how many days for market time
λ::Float64 = 1; # risk scale

### Implementation
Next, build the `world(...)` function. The `world(...)` function takes the action vector `a::Vector{Int64}` where the elements of `a::Vector{Int64}` are binary variables indicating whether to select an asset (`1`) or not (`0`). The length of the action vector `a` is $|\dim\mathcal{P}|$, i.e., the total number of assets available for selection.

> __What's in the action vector?__ The action vector `a::Vector{Int64}` is a binary vector of length $|\dim\mathcal{P}|$ where each element indicates whether to include (`1`) or exclude (`0`) the corresponding asset from the portfolio. For example, if we have 5 assets and the action vector is `[1, 0, 1, 0, 1]`, it means we are selecting assets 1, 3, and 5 for inclusion in the portfolio while excluding assets 2 and 4.

The `world(...)` function returns the portfolio Utility, the optimal number of shares for each selected asset, the fill price for each selected asset, and the user preference values $\gamma_i$ for each selected asset. 

In [24]:
function world(t::Int, a::Vector{Int64}, context::MyDynamicBanditPortfolioAllocationContextModel)

    # initialize -
    total_number_of_assets = context.number_of_assets; # total number of assets that we *could* purchase
    bounds = context.bounds; # bounds for how much we purchase
    B = context.B; # budget for this period
    mylocaltickers = context.tickers; # tickers for the assets in this portfolio
    localdataset = context.dataset; # dataset for the assets in this portfolio
    singleindexmodels = context.singleindexmodels; # single index models for the assets in this portfolio
    X̄ = context.X̄;
    number_of_samples_to_draw = context.number_of_samples_to_draw;

    # what is the min share purchase?
    min_share_purchase = bounds[1,1];

    # Compute the fill price vector -
    pₒ = zeros(total_number_of_assets);
    for i ∈ eachindex(mylocaltickers)
        ticker = mylocaltickers[i];
        H = localdataset[ticker][t,:high];
        L = localdataset[ticker][t,:low];
        f = rand();
        pₒ[i] = f*H + (1-f)*L; # randomness in the fill price
    end

    # next, compute the preference parameters γ -
    γ = zeros(total_number_of_assets);
    for i ∈ eachindex(mylocaltickers)
        ticker = mylocaltickers[i];

        # get the model -
        simmodel = singleindexmodels[ticker];
        α = simmodel.alpha;
        β = simmodel.beta;
        CI_alpha_L = simmodel.alpha_95_CI_lower;
        CI_alpha_U = simmodel.alpha_95_CI_upper;
        CI_beta_L = simmodel.beta_95_CI_lower;
        CI_beta_U = simmodel.beta_95_CI_upper;

        # compute random parameters -
        # αᵢ = rand(Uniform(CI_alpha_L, CI_alpha_U)); # sample from the confidence interval
        # βᵢ = rand(Uniform(CI_beta_L, CI_beta_U));
        αᵢ = α; # use the mean value
        βᵢ = β; # use the mean value

        # Compute the alpha and beta values -
        R = αᵢ + βᵢ*μₘ; # draw random value from the error distribution -
        γ[i] = tanh_fast(R/(βᵢ^λ));
    end

    # Compute the optimal share count -
    n = zeros(total_number_of_assets); # initialize space for optimal solution
    S = findall(aᵢ -> aᵢ == 1, a); # Which assets does our bandit want us to buy?
    number_of_selected_arms = length(S);

    # In the set of assets to explore, do we have any non-preferred assets?
    negative_gamma_flag = any(γ[S] .< 0);
    if (negative_gamma_flag == false)
        
        # easy case: all of my potential assets are preferred.
        γ̄ = sum(γ[S]);
        B̄ = B;
        for s ∈ S
            n[s] = (γ[s]/γ̄)*(B̄/pₒ[s]);
        end
    else

        # hard case: some assets are *not* preferred. 
        
        # Prep work for non-preferred case
        # First: the non-preferred assets are min_share_purchase -
        # Second: Compute the adjusted budget
        # Third: Compute γ̄
        B̄ = B;
        γ̄ = 0.0;
        for s ∈ S
            if (γ[s] < 0.0)
                B̄ += -min_share_purchase*pₒ[s];
                n[s] = min_share_purchase;
            else
                γ̄ += γ[s];
            end
        end

        # compute the optimal preferred assets -
        for s ∈ S
            if (γ[s] ≥ 0.0)
                n[s] = (γ[s]/γ̄)*(B̄/pₒ[s]);
            end
        end
    end

    # premultiplier -
    κ = negative_gamma_flag == true ? -1.0 : 1.0;
    
    # Compute the optimal utility -
    U = κ;
    for s ∈ S
        U *= (n[s]^γ[s])
    end
    
    # Return the utility and the share count that we allocated
    return U, n, pₒ, γ
end;

___

## Task 1: Maximum Utility Portfolio Optimization Problem
In this task, we'll solve the maximum utility portfolio optimization problem for a given set that we select such that we have the maximum __investor satisfaction__, i.e., maximum Utility. Let's start by specifying the possible tickers that our online stochastic bandit algorithm can choose from. We'll store these tickers in the `my_test_portfolio_tickers::Array{String,1}` variable:

In [27]:
my_test_portfolio_tickers = ["AAPL", "MSFT", "INTC", "MU", "AMD", "GS", "BAC", "WFC", "C", "F", "GM", 
    "JNJ", "PG", "UPS", "COST", "TGT", "WMT", "MRK", "PFE", "ADBE"]; # tickers selected for portfolio

In [28]:
length(my_test_portfolio_tickers) # how many tickers do we have?

20

Given this set of possible tickers, let's compute the expected growth rate vector and the covariance matrix for these assets using their single index model parameters. We don't use these values in the bandit calculation, but we will need them to compute the Sharpe ratio of the resulting portfolio later. 

We'll store these in the `μ̂_sim::Array{Float64,1}` and `Σ̂_sim::Array{Float64,2}` variables:

In [30]:
μ̂_sim, Σ̂_sim = let

    # initialize -
    N = length(my_test_portfolio_tickers); # number of assets in portfolio
    μ_sim = Array{Float64,1}(); # drift vector
    Σ̂_sim = Array{Float64,2}(undef, N, N); # covariance matrix for *our* portfolio
    
    # Ḡₘ = mean(Gₘ); # mean of market factor # TODO: already loaded above, we'll use 2014 - 2024 data value
    σ²ₘ = Varₘ; # variance of market factor # TODO: already loaded above, we'll use 2014 - 2024 data value

    # compute the expected growth rate (return) for each of our tickers -
    for i ∈ eachindex(my_test_portfolio_tickers)
        ticker = my_test_portfolio_tickers[i];
        data = sim_model_parameters[ticker]; # get the data for this ticker
        αᵢ = data.alpha; # get alpha
        βᵢ = data.beta; # get beta
        Ḡᵢ = αᵢ + βᵢ* Ḡₘ; # compute the growth rate for this ticker
        push!(μ_sim, Ḡᵢ); # append growth rate value to μ_sim
    end

    # compute the covariance matrix using the single index model -
    for i ∈ eachindex(my_test_portfolio_tickers)

        ticker_i = my_test_portfolio_tickers[i];
        data_i = sim_model_parameters[ticker_i]; # get the data for ticker i
        βᵢ = data_i.beta; # get beta for ticker i
        σ²_εᵢ = (Δt)*data_i.training_variance; # residual variance for ticker i

        for j ∈ eachindex(my_test_portfolio_tickers)
            
            ticker_j = my_test_portfolio_tickers[j];
            data_j = sim_model_parameters[ticker_j]; # get the data for ticker j
            βⱼ = data_j.beta; # get beta for ticker j
            σ²_εⱼ = (Δt)*data_j.training_variance; # residual variance for ticker j
            
            if i == j
                Σ̂_sim[i,j] = βᵢ*βⱼ*σ²ₘ + σ²_εᵢ; # diagonal elements
            else
                Σ̂_sim[i,j] = βᵢ*βⱼ*σ²ₘ; # off-diagonal elements
            end
        end
    end

    (μ_sim, Σ̂_sim*Δt); # return
end;

To implement our bandit strategy, we first build the epsilon-greedy dynamic noise algorithm model. This model holds some information about the problem, e.g., the number of assets available for selection, and the learning rate $\alpha$. We store this model in the `dynamic_algorithm_model::MyEpsilonGreedyDynamicNoiseAlgorithmModel` variable:

In [32]:
dynamic_algorithm_model = let

    # initialize -
    algorithm = nothing; # Initialize the algorithm variable to nothing; this variable will be used to store the algorithm model
    K = length(my_test_portfolio_tickers); # TODO: Number of tickers to consider
    
    # TODO: Build an algorithm model by uncommenting the code block below
    algorithm = build(MyEpsilonGreedyDynamicNoiseAlgorithmModel, (
        K = K, # arms 
        α = α, # learning rate
    ));

    # return the algorithm -
    algorithm;
end;

Next, we set bounds on the number of shares that we can hold for each asset in our portfolio. The lower bound `ϵ::Float64` represents the __minimum number of shares__ required for portfolio inclusion, while the upper bound is unbounded to allow for flexible allocation. 

> __Why a non-zero lower bound?__ Setting a non-zero lower bound `ϵ::Float64` follows from the form of the Cobb-Douglas utility function, if any asset has zero allocation, the overall utility becomes zero which does not reflect realistic investor behavior. Thus, we set a small positive value for `ϵ::Float64` to ensure that each selected asset contributes positively to the portfolio's utility.

We'll store this in the `static_share_bounds::Array{Float64,2}` variable:

In [34]:
static_share_bounds = let

    # initialize -
    K = length(my_test_portfolio_tickers); # TODO: Number of tickers to consider
    bounds = Array{Float64,2}(undef, K, 2);
    
    # build my bounds array -
    for i ∈ eachindex(my_test_portfolio_tickers)
        bounds[i,1] = ϵ; # min shares that we can hold of this asset
        bounds[i,2] = Inf; # max number of shares, Inf says this is unbounded
    end
    bounds;
end;

Now we build the context model that encapsulates all the data and parameters needed for the bandit algorithm to make portfolio decisions. The context includes the market data, single index model parameters, ticker symbols, budget constraints, and bounds on share purchases. We'll store this in the `dynamic_context_model::MyDynamicBanditPortfolioAllocationContextModel` variable:

In [36]:
dynamic_context_model = let

    # initialize -
    K = length(my_test_portfolio_tickers); # TODO: Number of tickers to consider
    nₒ = ones(K); # initial guess, assume 1 x share for each

    # build -
    contextmodel = build(MyDynamicBanditPortfolioAllocationContextModel, (
        dataset = dataset,
        singleindexmodels = sim_model_parameters,
        tickers = my_test_portfolio_tickers,
        nₒ = nₒ,
        B = B,
        bounds = static_share_bounds,
        X̄ = X̄,
        R̄ₘ = Ḡₘ,
        number_of_samples_to_draw = size(X̄,2),
        μₒ = zeros(2^K) # initially we have *no* idea which arm is best
    ));

    contextmodel;
end;

Finally, we test our approach by solving the utility maximization problem for the case where we select all available assets (by setting the action vector `a::Vector{Int64}` to all ones). This gives us a baseline utility `U::Float64` we could achieve with our full portfolio. 

> __Expectation:__ If this doesn't blow up, then it will return a finite utility value, optimal share counts, fill prices, and preference parameters for each asset in the portfolio. Then we can move on to implementing the bandit algorithm to select subsets of assets dynamically.

So what do we see?

In [38]:
(U, n, pₒ, γ) = let
   
    # call my world function to test -
    K = length(my_test_portfolio_tickers); # TODO: Number of tickers to consider
    t = 1; # what time period are we in?

    # call the world function directly -
    U, n, pₒ, γ = world(t, ones(Int64,K), dynamic_context_model);
end;

In [39]:
U

-17.680530283780854

__Preferred versus Non-Preferred Assets:__ The `γ::Array{Float64,1}` vector computed above indicates whether an asset is preferred or non-preferred. If $\gamma_i \geq 0$, then the asset is preferred; otherwise, it is non-preferred. Let's look at what assets are preferred versus non-preferred:

In [41]:
let

    # initialize -
    df = DataFrame();

    for i ∈ eachindex(my_test_portfolio_tickers)
        ticker = my_test_portfolio_tickers[i];

        # get the model -
        row_df = (
            ticker = ticker,
            γᵢ = γ[i],
            preferred = γ[i] >= 0 ? "Yes" : "No",

        );
        push!(df, row_df);
    end

    # make a table -
     pretty_table(df, backend = :text, fit_table_in_display_vertically = false, fit_table_in_display_horizontally = false,
         table_format = TextTableFormat(borders = text_table_borders__compact)
    );
end

 -------- ------------ -----------
  ticker           γᵢ   preferred 
  String      Float64      String 
 -------- ------------ -----------
    AAPL      0.19245         Yes
    MSFT     0.190485         Yes
    INTC   -0.0192358          No
      MU    0.0745648         Yes
     AMD     0.177492         Yes
      GS    0.0810726         Yes
     BAC    0.0659029         Yes
     WFC    0.0322735         Yes
       C     0.017186         Yes
       F   -0.0283554          No
      GM     0.018269         Yes
     JNJ    0.0759124         Yes
      PG     0.135188         Yes
     UPS    0.0203514         Yes
    COST     0.255981         Yes
     TGT    0.0826703         Yes
     WMT     0.230516         Yes
     MRK     0.111904         Yes
     PFE   -0.0210359          No
    ADBE     0.144215         Yes
 -------- ------------ -----------


Now that we have the preferred versus non-preferred assets, we'll make a table that summarizes the resulting shares `n::Array{Float64,1}` purchased for each asset in our test portfolio:

In [43]:
let

    # initialize -
    df = DataFrame();
    CS = 0;
    t = 1;
    K = length(my_test_portfolio_tickers); # TODO: Number of tickers to consider
    context = dynamic_context_model;
    total_number_of_assets = context.number_of_assets; # total number of assets that we *could* purchase
    bounds = context.bounds; # bounds for how much we purchase
    B = context.B; # budget for this period
    mylocaltickers = context.tickers; # tickers for the assets in this portfolio
    localdataset = context.dataset; # dataset for the assets in this portfolio
    singleindexmodels = context.singleindexmodels; # single index models for the assets in this portfolio
    N = 2^K; # number of possible portfolios
    

    for i ∈ 1:K

        CS += n[i]*pₒ[i];
        wᵢ = (n[i]*pₒ[i]/B);

        row_df = (
            ticker = my_test_portfolio_tickers[i],
            Sᵢ = pₒ[i],
            γᵢ = γ[i],
            nᵢ = n[i],
            wᵢ = wᵢ,
            UC = n[i]*pₒ[i],
            CS = CS,
        );
        push!(df, row_df);
    end

    pretty_table(df, backend = :text, fit_table_in_display_vertically = false, fit_table_in_display_horizontally = false,
         table_format = TextTableFormat(borders = text_table_borders__compact)
    );
end

 -------- --------- ------------ ---------- ------------ ------------ ---------
  ticker        Sᵢ           γᵢ         nᵢ           wᵢ           UC        CS 
  String   Float64      Float64    Float64      Float64      Float64   Float64 
 -------- --------- ------------ ---------- ------------ ------------ ---------
    AAPL   243.901      0.19245    4.13886     0.100947      1009.47   1009.47
    MSFT   422.273     0.190485    2.36616    0.0999163      999.163   2008.63
    INTC    20.351   -0.0192358      0.001    2.0351e-6     0.020351   2008.65
      MU   89.6988    0.0745648    4.36037     0.039112       391.12   2399.77
     AMD   125.358     0.177492    7.42684    0.0931013      931.013   3330.79
      GS   573.093    0.0810726   0.742036    0.0425256      425.256   3756.04
     BAC    44.628    0.0659029    7.74593    0.0345685      345.685   4101.73
     WFC   71.2661    0.0322735    2.37541    0.0169286      169.286   4271.01
       C   69.9792     0.017186     1.2882   0.0

## Task 2: Solve the Online Bandit Portfolio Optimization Problem
In this task, we use the epsilon-greedy bandit algorithm to explore different portfolio combinations and learn which asset subsets maximize investor utility. 

The algorithm iteratively selects actions (portfolio compositions), observes the resulting utility, and updates its beliefs about the reward distribution. The results are stored in the return variables `R::Array{Float64,2}`, `μ::Array{Float64,1}`, `S::Array{Float64,2}`, `P::Array{Float64,2}`, and `G::Array{Float64,2}`. 

Let's run the algorithm for 250 iterations:

In [45]:
R, μ, S, P, G = my_bandit_solve(dynamic_algorithm_model, T = 250, world = world, context = dynamic_context_model); # check the world function

In [82]:
μ

1048576-element Vector{Float64}:
  0.00817706199372983
  0.007316749594698126
  0.011463189416684382
 -0.0045684346124372525
 -0.009335995326710544
 -0.008355953015113195
 -0.013105331266013598
  0.0056913080644323145
  0.009939864282972334
  0.00888247402480798
  0.013301495970579164
 -0.0065000176556160695
 -0.01134377101156865
  ⋮
 -0.06880751363855016
 -0.06201797537041391
 -0.068674582639809
 -0.05645906117588358
 -0.06315712319684953
 -0.05671323278186551
 -0.06208232244254034
 -0.06461193107600219
 -0.07200624117430926
 -0.0648214298710901
 -0.07107903359637201
 -0.0714223538343581

In [46]:
S # share counts over for each trial

250×20 Matrix{Float64}:
  4.17194  2.35732  0.001   4.37683  …  13.4067   5.92771  0.001  1.76777
  8.54755  4.9024   0.0     0.0         27.4832   0.0      0.001  3.64952
 12.1721   6.94812  0.0    12.7076       0.0     17.2643   0.0    0.0
  5.95207  3.40023  0.0     0.0         19.1009   0.0      0.001  2.54983
  0.0      5.26682  0.001   0.0         29.6582   0.0      0.001  3.89946
  0.0      4.76169  0.0     8.86755  …   0.0     11.8672   0.001  0.0
 12.1155   6.95415  0.0    12.7325       0.0     17.337    0.0    0.0
  0.0      0.0      0.0    12.6491       0.0     17.1256   0.0    5.05223
  7.29385  4.14381  0.001   7.64012      0.0     10.4301   0.001  3.1182
  0.0      4.26716  0.0     0.0         23.865    0.0      0.0    3.17578
  7.79623  0.0      0.001   8.20461  …  24.9815  11.1003   0.0    3.3305
  0.0      0.0      0.0     9.62205     28.8664   0.0      0.001  3.79996
  0.0      0.0      0.0     0.0         28.5782  12.6513   0.001  0.0
  ⋮                             

We can sort the possible portfolio compositions by the estimated average utility `μ::Array{Float64,1}` to see which portfolios the bandit algorithm thinks are best:

In [48]:
sorted_reward_index_array = sortperm(μ, rev=true) # sort the index of the rewards

1048576-element Vector{Int64}:
  770419
  130506
  649273
  744883
  583970
    5584
   80145
  669801
  642114
  163851
  235739
  235995
  236763
       ⋮
   61079
  253899
  987282
  355567
  770621
  778118
  514855
  892185
  237661
  205531
  991205
 1020415

__What's in the best portfolio?__  
Let's build a table that shows the optimal portfolio composition identified by the bandit algorithm after 250 periods of exploration. 

The utility value `U::Float64` indicates the investor satisfaction level for this portfolio, while the weights `wᵢ::Float64` show the proportion of budget allocated to each asset. Notice that the bandit algorithm may have selected a portfolio subset (where some `nᵢ::Float64` values are near zero) rather than including all assets, demonstrating how the algorithm balances portfolio complexity with investor utility.

In [50]:
let

    # initialize -
    df = DataFrame();
    CS = 0;
    t = 1;
    K = length(my_test_portfolio_tickers); # TODO: Number of tickers to consider
    context = dynamic_context_model;
    total_number_of_assets = context.number_of_assets; # total number of assets that we *could* purchase
    bounds = context.bounds; # bounds for how much we purchase
    B = context.B; # budget for this period
    mylocaltickers = context.tickers; # tickers for the assets in this portfolio
    localdataset = context.dataset; # dataset for the assets in this portfolio
    singleindexmodels = context.singleindexmodels; # single index models for the assets in this portfolio
    N = 2^K; # number of possible portfolios

    # compute the best portfolio -
    j = sorted_reward_index_array[1]; # get a portfolio index 
    a = digits(j, base=2, pad=K); # generate a binary representation of the number, with K digits  
    if (j == N)
        a = digits(j - 1, base=2, pad=K); # generate a binary representation of the number, with K digits  
    end

    # call world -
    U, n, pₒ, γ = world(t, a, dynamic_context_model);

    @show U,j

    for i ∈ 1:K

        CS += n[i]*pₒ[i];
        wᵢ = (n[i]*pₒ[i]/B);

        row_df = (
            ticker = my_test_portfolio_tickers[i],
            Sᵢ = pₒ[i],
            γᵢ = γ[i],
            nᵢ = n[i],
            wᵢ = wᵢ,
            UC = n[i]*pₒ[i],
            CS = CS,
        );
        push!(df, row_df);
    end

     pretty_table(df, backend = :text, fit_table_in_display_vertically = false, fit_table_in_display_horizontally = false,
         table_format = TextTableFormat(borders = text_table_borders__compact)
    );
end

(U, j) = (10.02559152367366, 770419)
 -------- --------- ------------ ---------- ----------- --------- ---------
  ticker        Sᵢ           γᵢ         nᵢ          wᵢ        UC        CS 
  String   Float64      Float64    Float64     Float64   Float64   Float64 
 -------- --------- ------------ ---------- ----------- --------- ---------
    AAPL   242.833      0.19245    5.11345    0.124171   1241.71   1241.71
    MSFT   420.042     0.190485    2.92598    0.122903   1229.03   2470.75
    INTC   20.1746   -0.0192358        0.0         0.0       0.0   2470.75
      MU   88.0118    0.0745648        0.0         0.0       0.0   2470.75
     AMD   122.266     0.177492    9.36648     0.11452    1145.2   3615.95
      GS   573.372    0.0810726   0.912307   0.0523091   523.091   4139.04
     BAC     44.15    0.0659029    9.63113   0.0425214   425.214   4564.26
     WFC   70.3486    0.0322735        0.0         0.0       0.0   4564.26
       C   70.2541     0.017186    1.57836   0.0110886   11

Ok, so what is the Sharpe ratio of this portfolio?

In [52]:
let

    # initialize -
    t = 1;
    K = length(my_test_portfolio_tickers); # TODO: Number of tickers to consider
    N = 2^K; # number of possible portfolios

    # compute the best portfolio -
    j = sorted_reward_index_array[1]; # get a portfolio index 
    a = digits(j, base=2, pad=K); # generate a binary representation of the number, with K digits  
    if (j == N)
        a = digits(j - 1, base=2, pad=K); # generate a binary representation of the number, with K digits  
    end

    # call world -
    U, n, pₒ, γ = world(t, a, dynamic_context_model);

    # compute the weights -
    w = zeros(K);
    for i ∈ 1:K
        w[i] = (n[i]*pₒ[i])/B;
    end

    # get the alpha and beta values for each asset in the portfolio -
    α_vector = zeros(K);
    β_vector = zeros(K);

    # compute the alpha for the portfolio -
   for i ∈ 1:K
       ticker = my_test_portfolio_tickers[i];
       α_vector[i] = sim_model_parameters[ticker].alpha;
       β_vector[i] = sim_model_parameters[ticker].beta;
   end

   # compute the alpha and beta for the portfolio -
   α_portfolio = dot(w, α_vector);
   β_portfolio = dot(w, β_vector);
   numerator = α_portfolio + β_portfolio*μₘ - risk_free_rate;
   denominator = sqrt(transpose(w)*Σ̂_sim*w);
   sharpe_ratio = numerator/denominator;

   println("The Sharpe ratio of this portfolio is: $(sharpe_ratio)");
end

The Sharpe ratio of this portfolio is: 0.8508082016366805


## Summary
This example demonstrates how online stochastic bandit algorithms can be effectively applied to solve the combined asset selection and weight allocation problems in portfolio management through the integration of epsilon-greedy exploration with utility maximization.

> __Key Takeaways:__
> 
> * __Bandit Algorithms for Portfolio Selection:__ Multi-armed bandit algorithms like epsilon-greedy enable adaptive asset selection by balancing exploration of uncertain portfolios with exploitation of known high-utility combinations, avoiding the need to exhaustively evaluate all possible subsets.
> * __Utility Maximization for Weight Allocation:__ Once a portfolio subset is selected, Cobb-Douglas utility functions provide a principled approach to weight allocation that reflects investor preferences and risk tolerance, with the preference parameters `γ::Array{Float64,1}` distinguishing between preferred and non-preferred assets.
> * __Integration of Selection and Allocation:__ The combination of bandit algorithms for asset selection with Cobb-Douglas utility maximization for weight allocation provides a theoretically sound and computationally efficient framework for simultaneously addressing both problems in portfolio optimization.

By combining these techniques, investors can manage both the complexity of asset selection and the precision of weight allocation in a computationally efficient and theoretically sound framework.
___

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. You should decide solely based on your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.